In [19]:
from tqdm import tqdm
import argparse
import os
import keras.backend as K
import keras.layers as KL
import numpy as np
from keras.callbacks import TensorBoard
from keras.datasets import mnist
from keras import utils
from keras import Input, Model

def prepare_data():
    """
    prepare mnist data
    """
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
    X_train = X_train.astype(np.float32) / 255.
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))
    X_test = X_test.astype(np.float32) / 255.

    y_train, y_test = utils.to_categorical(y_train, 10), utils.to_categorical(y_test, 10)

    return (X_train, y_train), (X_test, y_test)

In [2]:
class LeNet:
    """
    Before calling mc-dropout, use 'set_mc_dropout_rate' 
    """
    def __init__(self, input_shape, num_classes):
        self.mc_dropout_rate = K.variable(value=0)  # dropout before the last fully connected layer
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.model = self._build_model()
        
    def _build_model(self):
        inp = Input(shape=self.input_shape)
        x = KL.Conv2D(filters=20, kernel_size=5, strides=1)(inp)
        x = KL.MaxPool2D(pool_size=2, strides=2)(x)
        x = KL.Conv2D(filters=50, kernel_size=5, strides=1)(x)
        x = KL.MaxPool2D(pool_size=2, strides=2)(x)
        x = KL.Flatten()(x)
        x = KL.Dense(500, activation='relu')(x)
        x = KL.Lambda(lambda x: K.dropout(x, level=self.mc_dropout_rate))(x)  # dropout before the last fully connected layer
        x = KL.Dense(self.num_classes, activation='softmax')(x)

        return Model(inputs=inp, outputs=x, name='lenet-mc-dropout')
    
    def set_mc_dropout_rate(self, new_rate):
        K.set_value(self.mc_dropout_rate, new_rate)

In [8]:
def evalute_mc(net, X_test, y_test, mc_dropout_rate, sample_times=50):
    net.set_mc_dropout_rate(mc_dropout_rate)
    model = net.model
    batch_size = 1000
    err = 0.
    for batch_id in tqdm(range(X_test.shape[0] // batch_size)):
        # take batch of data
        x = X_test[(batch_id*batch_size):((batch_id + 1)*batch_size)]
        # init empty predictions
        y_ = np.zeros((sample_times, batch_size, y_test[0].shape[0]))

        for sample_id in range(sample_times):
            # save predictions from a sample pass
            y_[sample_id] = model.predict(x, batch_size)

        # average over all passes
        mean_y = y_.mean(axis=0)
        # evaluate against labels
        y = y_test[batch_size * batch_id: (batch_id + 1) * batch_size]
        # compute error
        err += np.count_nonzero(np.not_equal(mean_y.argmax(axis=1), y.argmax(axis=1)))

    err = err / X_test.shape[0]

    return 1. - err

In [20]:
def mc_dropout(net, X_test, batch_size=1000, dropout=0.5, T=100):
    net.set_mc_dropout_rate(dropout)
    model = net.model
    repititions = []
    for i in tqdm(range(T)):
        pred = model.predict(X_test, batch_size)
        repititions.append(pred)
    net.set_mc_dropout_rate(0)

    repititions = np.array(repititions)
    mc = np.var(repititions, 0)
    mc = np.mean(mc, -1)
    return -mc

In [5]:
batch_size = 32
epochs = 2 

In [6]:
# load the data
(X_train, y_train), (X_test, y_test) = prepare_data()

# prepare the model
lenet = LeNet(
    input_shape=X_train.shape[1:],
    num_classes=10,
)


#### train
model = lenet.model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# train the network
model.fit(
    x=X_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 17s 289us/step - loss: 0.1198 - acc: 0.9634 - val_loss: 0.0621 - val_acc: 0.9800
Epoch 2/2
60000/60000 [==============================] - 17s 287us/step - loss: 0.0495 - acc: 0.9848 - val_loss: 0.0391 - val_acc: 0.9875


In [10]:
acc = evalute_mc(lenet, X_test, y_test, mc_dropout_rate=0.5)
print("acc = ", acc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:19<00:00,  1.97s/it]


acc =  0.9872


In [44]:
wtf = np.random.rand(*X_test[:1].shape)

In [45]:
ok_socre = mc_dropout(lenet, X_test[:1])
wtf_score = mc_dropout(lenet, wtf)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 1671.12it/s]


In [46]:
print(ok_socre)
print(wtf_score)

[-4.2645273e-10]
[-0.01651107]
